<a href="https://colab.research.google.com/github/tomoyahiroe/transformers-playground/blob/main/how_tokenizer_insert_special_token.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
# transformersのインストール
!pip install transformers[ja,sentencepiece,torch]

In [85]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoModelForQuestionAnswering
import pprint
import torch

## どのようにTokenizerは特殊トークンを挿入しているのか

### cl-tohoku/bert-base-japanese-v3の例

In [86]:
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v3")
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

- 文頭と文末に特殊トークンが挿入される例

In [87]:
text = "今日は良い天気ですね。"
encoded_inputs = tokenizer(text, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

['[CLS]', '今日', 'は', '良い', '天気', 'です', 'ね', '。', '[SEP]']


- 複数の文章をひとまとまりの文字列として入力しても、途中に`[SEP]`は挿入されない

In [88]:
text = "今日は良い天気ですね。公園で本でも読みましょうか"
encoded_inputs = tokenizer(text, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

['[CLS]', '今日', 'は', '良い', '天気', 'です', 'ね', '。', '公園', 'で', '本', 'で', 'も', '読み', 'ましょう', 'か', '[SEP]']


- 一文毎に分割して配列にして渡せば、`[SEP]`が入力されるが、`[CLS]`も入力される

In [89]:
text = ["今日は良い天気ですね。","公園で本でも読みましょうか"]
encoded_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# input_ids をトークンにデコードして確認します
for i, input_id_list in enumerate(encoded_inputs["input_ids"]):
    decoded_tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    print(f"original text: {text[i]}")
    print(f"model input(decoded): {decoded_tokens}")
    print(f"attention mask: {encoded_inputs['attention_mask'][i].tolist()}")
    print("-" * 20)

original text: 今日は良い天気ですね。
model input(decoded): ['[CLS]', '今日', 'は', '良い', '天気', 'です', 'ね', '。', '[SEP]', '[PAD]']
attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
--------------------
original text: 公園で本でも読みましょうか
model input(decoded): ['[CLS]', '公園', 'で', '本', 'で', 'も', '読み', 'ましょう', 'か', '[SEP]']
attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
--------------------


- tokenizerの引数に二つ入力することで、`[SEP]` のみを挿入できる

In [90]:
text1 = "今日は良い天気ですね。"
text2 = "公園で本でも読みましょうか"

In [91]:
encoded_inputs = tokenizer(text1, text2, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

['[CLS]', '今日', 'は', '良い', '天気', 'です', 'ね', '。', '[SEP]', '公園', 'で', '本', 'で', 'も', '読み', 'ましょう', 'か', '[SEP]']


## abeja/gpt2-large-japaneseの例

In [92]:
tokenizer = AutoTokenizer.from_pretrained("abeja/gpt2-large-japanese")
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [93]:
text = "日本の首都は"
encoded_inputs = tokenizer(text, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)
pprint.pp(encoded_inputs)

['▁日本', 'の首都', 'は', '</s>']
{'input_ids': tensor([[  491, 11308,    15,     2]]),
 'attention_mask': tensor([[1, 1, 1, 1]])}


In [94]:
text = "アメリカの首都はワシントンD.C.です。日本の首都は"
encoded_inputs = tokenizer(text, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

['▁アメリカの', '首都', 'は', 'ワシントン', 'D', '.', 'C', '.', 'です', '。', '日本の', '首都', 'は', '</s>']


In [95]:
text = ["アメリカの首都はワシントンD.C.です。", "日本の首都は"]
encoded_inputs = tokenizer(text1, text2, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

['▁', '今日', 'は', '良い', '天気', 'です', 'ね', '。', '</s>', '▁', '公園', 'で', '本', 'でも', '読み', 'ましょう', 'か', '</s>']


In [96]:
text1 = "アメリカの首都はワシントンD.C.です。"
text2 = "日本の首都は"
encoded_inputs = tokenizer(text1, text2, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

['▁アメリカの', '首都', 'は', 'ワシントン', 'D', '.', 'C', '.', 'です', '。', '</s>', '▁日本', 'の首都', 'は', '</s>']


## 文章を複数入力したい２つのシチュエーションについて

1. 一つの出力を得たいが、BERTに文章を区別してもらいたい

2. 一つ一つの文章毎に、BERTに特定のタスクを行ってもらいたい

In [97]:
model = AutoModelForCausalLM.from_pretrained("abeja/gpt2-large-japanese")

### 一つの出力を得たい

#### tokenizerの引数に２つの文字列を渡すパターン

In [98]:
context = "アメリカの首都はワシントンD.C.です。"
question = "そして、日本の首都は"

In [99]:
encoded_inputs = tokenizer(context, question, return_tensors="pt")
decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0])
print(decoded_tokens)

['▁アメリカの', '首都', 'は', 'ワシントン', 'D', '.', 'C', '.', 'です', '。', '</s>', '▁そして', '、', '日本の', '首都', 'は', '</s>']


In [100]:
output = model.generate(**encoded_inputs, max_length=20, pad_token_id=tokenizer.pad_token_id)

In [101]:
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)
print(decoded_output)

['アメリカの首都はワシントンD.C.です。 そして、日本の首都は首都機能が']


#### tokenizerに配列を渡すパターン

In [102]:
context = "アメリカの首都はワシントンD.C.です。"
question = "そして、日本の首都は"
text = [context, question]

In [103]:
encoded_inputs = tokenizer(text, return_tensors="pt", padding=True)
for i, input_id_list in enumerate(encoded_inputs["input_ids"]):
    decoded_tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    print(f"original text: {text[i]}")
    print(f"model input(decoded): {decoded_tokens}")
    print(f"attention mask: {encoded_inputs['attention_mask'][i].tolist()}")
    print("-" * 20)

original text: アメリカの首都はワシントンD.C.です。
model input(decoded): ['▁アメリカの', '首都', 'は', 'ワシントン', 'D', '.', 'C', '.', 'です', '。', '</s>']
attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
--------------------
original text: そして、日本の首都は
model input(decoded): ['▁そして', '、', '日本の', '首都', 'は', '</s>', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
attention mask: [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
--------------------


In [104]:
output = model.generate(**encoded_inputs, max_length=20, pad_token_id=tokenizer.pad_token_id)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [105]:
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)
print(decoded_output)

['アメリカの首都はワシントンD.C.です。ワシントンD.C.は、アメリカ合衆国の首都', 'そして、日本の首都は東京 ですが、 首都は ']
